In [1]:
import os
import math


from itertools import permutations

from kneed import KneeLocator

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (silhouette_score, adjusted_rand_score)

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
WORKDIR = "" #os.path.abspath(os.getcwd())

YEARS_AVAILABLE = [2014, 2018, 2021]

## Get the data

__Load into Pandas DataFrame__

In [3]:
file = ''.join([WORKDIR, "../../data/output/hotspot_spi.csv"]) 
dataset = pd.read_csv(file)
dataset

,ano,estado,municipio,diasemchuva,precipitacao,riscofogo,frp,fococalor,riscofogocat,IPS Amazônia,...,Gravidez na infância e adolescência,Trabalho Infantil,Vulnerabilidade familiar,Violência contra indígenas,Violência contra indígenas Taxa,Violência contra mulheres,Violência infantil,Violência infantil Taxa,Empregos ensino superior,Mulheres com empregos ensino superior
0,2014,ACRE,ACRELANDIA,0.9,0.38,0.23,0.00,441,BAIXO,54.64,...,26.77,288.91,12.27,0.00,1.0,29.60,22.32,2.0,13.81,7.35
1,2014,ACRE,ASSIS BRASIL,1.7,0.46,0.15,0.00,327,MUITO_BAIXO,53.48,...,36.22,201.25,17.35,1.88,2.0,58.91,80.48,3.0,8.17,4.54
2,2014,ACRE,BRASILEIA,1.4,0.36,0.19,0.00,1120,BAIXO,50.05,...,31.08,144.83,20.11,40.00,5.0,300.00,683.19,5.0,9.97,6.59
3,2014,ACRE,BUJARI,0.8,0.36,0.12,0.00,339,MUITO_BAIXO,54.02,...,34.30,824.47,16.12,0.00,1.0,45.11,32.66,2.0,12.97,9.16
4,2014,ACRE,CAPIXABA,0.8,0.10,0.08,0.00,455,MUITO_BAIXO,53.15,...,32.05,172.42,26.07,0.00,1.0,0.00,0.00,1.0,11.60,7.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2311,2021,TOCANTINS,TOCANTINOPOLIS,23.2,0.32,0.85,9.03,799,ALTO,53.76,...,25.06,19.09,58.55,16.50,4.0,632.00,433.17,5.0,15.30,11.06
2312,2021,TOCANTINS,TUPIRAMA,39.5,0.94,0.81,7.53,180,ALTO,53.14,...,30.77,9.64,53.85,0.00,1.0,148.61,141.85,4.0,20.10,14.28
2313,2021,TOCANTINS,TUPIRATINS,43.5,0.41,0.89,9.99,690,ALTO,60.37,...,32.14,3.05,28.57,0.00,1.0,159.21,0.00,1.0,19.09,15.72
2314,2021,TOCANTINS,WANDERLANDIA,17.7,0.42,0.80,4.85,622,ALTO,54.31,...,27.59,11.80,81.03,0.00,1.0,0.00,24.26,2.0,15.15,12.15


In [4]:
labelEncoder = LabelEncoder()
dataset["estado"] = labelEncoder.fit_transform(dataset["estado"])
dataset["municipio"] = labelEncoder.fit_transform(dataset["municipio"])

## Functions

#### The Elbow Method

In [5]:
def peform_elbow_method(kmeans_kwargs, n_inter, features):
    sse = []
    min_cluster = 1
    
    for k in range(min_cluster, n_inter):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(features)
        sse.append(kmeans.inertia_)
    
    show_elbow_method(sse, n_inter, min_cluster)

        
def show_elbow_method(sse, n_inter, min_cluster):
    kl = KneeLocator(
        range(1, n_inter), 
        sse, 
        curve="convex", 
        direction="decreasing")
    
    plt.plot(range(1, n_inter), sse)
    
    plt.xticks(range(1, n_inter))
    plt.xlabel("Number of Clusters")
    plt.axvline(kl.elbow, c='tab:gray', ls='--')
    
    plt.ylabel("SSE")
    plt.show()

#### The Silhouette Coefficients

In [6]:
def peform_silhouette_coefficients(kmeans_kwargs, n_inter, features):
    #The silhouette score() function needs a minimum of two clusters, or it will raise an exception.
    min_cluster = 2
    sicoefs = []
    
    for k in range(min_cluster, n_inter):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(features)
        score = silhouette_score(features, kmeans.labels_)
        sicoefs.append(score)
        
    show_silhouette_coefficients(sicoefs, n_inter, min_cluster)

def show_silhouette_coefficients(sicoefs, n_inter, min_cluster):
    plt.plot(range(min_cluster, n_inter), sicoefs)
    plt.xticks(range(min_cluster, n_inter))
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Coefficient")
    plt.show()

#### Ajusted Rand Index

In [7]:
def perform_ari(true_labels, predicted_label):
    ari = adjusted_rand_score(true_labels, predicted_label)
    return round(ari, 2)

In [8]:
def xyz(features, true_labels, n_cluster=10, n_cluster_val=11):
    print("===========================================================")
    print("Features:\n", features[:5])
    print("True Labels:\n", true_labels[:5])
    print("===========================================================")
    
    
    # Standarization
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    print("Scaled Features:\n", scaled_features[:5])
    print("===========================================================")
    
    
    # Create KMeans Model
    kmeans = KMeans(n_clusters=n_cluster)
    _ = kmeans.fit(scaled_features)
    
    
    # show data points clusters
    y_pred = kmeans.fit_predict(scaled_features)
    plt.figure(figsize=(10, 10))
    plt.scatter(scaled_features[:, 0], scaled_features[:, 1], c=y_pred)
    plt.show()
    
    
    # Statistics from the initialization
    print("===========================================================")
    # The lowest SSE value
    print("Lowest SSE Value:", kmeans.inertia_)

    # The number of iterations required to converge
    print("Iterations Required to Converge:", kmeans.n_iter_)

    # Final locations of the centroid
    print("Final locations of the centroid:\n", kmeans.cluster_centers_)
    print("===========================================================")
    
        
    # Choosing the Appropriate Number of Clusters
    kmeans_kwargs = kmeans.get_params()
    del kmeans_kwargs['n_clusters']
    
    peform_elbow_method(kmeans_kwargs, n_cluster_val, scaled_features)
    peform_silhouette_coefficients(kmeans_kwargs, n_cluster_val, scaled_features)
    
    
    # Ajusted Rand Index
    print("===========================================================")
    print("Ajusted Rand Index:\n", perform_ari(true_labels, kmeans.labels_))
    print("===========================================================")
    
    return pd.DataFrame(
        {
            "true_labels": true_labels,
            "pred_labes": kmeans.labels_
        }
    )

In [9]:
def abc(features, true_labels, n_cluster=10, n_cluster_val=11):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=n_cluster)
    kmeans.fit(scaled_features)
    
    return perform_ari(true_labels, kmeans.labels_)

## Model Params

In [10]:
YEAR = 2021
N_CLUSTER = len(dataset.riscofogocat.unique())

print("Selected year:", YEAR)
print("Number of clusters:", N_CLUSTER)

Selected year: 2021
Number of clusters: 5


## 1º Study Case - Only the dimensions

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = dsyear["riscofogocat"].values
features = dsyear[[
    "Necessidades Humanas Básicas",
    "Fundamentos para o Bem-Estar",
    "Oportunidades"
]].values
xxx= xyz(features, true_labels, n_cluster=N_CLUSTER)


from pathlib import Path
path = Path(f"{os.path.abspath(os.getcwd())}/xxx.csv")
path.parent.mkdir(parents=True, exist_ok=True)
xxx.to_csv(path, index=False)



## 2º Study Case - "Necessidades Humanas Básicas" Components

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[[
    "Nutrição e cuidados médicos básicos",
    "Água e saneamento",
    "Moradia",
    "Segurança pessoal"
]].values

xyz(features, true_labels, n_cluster=N_CLUSTER)

## 3º Study Case - "Oportunidades" Components

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[[
    "Direitos individuais",
    "Liberdade individual e de escolha",
    "Tolerância e inclusão",
    "Acesso à educação superior"
]].values

xyz(features, true_labels, n_cluster=N_CLUSTER)

## 4º Study Case - "Necessidades Humanas Básicas" Indicators

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[[
    "Mortalidade infantil até 5 anos",
    "Mortalidade materna",
    "Mortalidade por desnutrição",
    "Mortalidade por doenças infecciosas",
    "Subnutrição",
    
    "Abastecimento de água",
    "Esgotamento sanitário",
    "Índice de atendimento de agua",
    
    "Coleta de lixo",
    "Moradias com iluminação adequada",
    "Moradias com parede adequada",
    "Moradias com piso adequado",
    
    "Assassinatos de jovens",
    "Assassinatos de jovens Taxa",
    "Homicídios",
    "Homicídios Taxa",
    "Mortes por acidente no trânsito",

]].values

xyz(features, true_labels, n_cluster=N_CLUSTER)

### 4º Study Case - "Fundamentos para o Bem-Estar" Indicators

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels =  LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[[
    "Abandono escolar ensino fundamental",
    "Distorção idade-série ensino fundamental",
    "Distorção idade-série ensino médio",
    "Qualidade da educação Ideb ensino fundamental",
    "Reprovação escolar ensino fundamental",
    
    "Densidade internet banda-larga",
    "Densidade telefonia fixa",
    "Densidade telefonia movel",
    "Densidade TV por assinatura",
    
    "Mortalidade por diabetes mellitus",
    "Mortalidade por câncer",
    "Mortalidade por doenças circulatórias",
    "Mortalidade por doenças respiratórias",
    "Mortalidade por suicídios",

    "Áreas Protegidas",
    "Desmatamento acumulado",
    "Desmatamento recente",
    "Emissões CO2",
    "Focos de calor por habitantes"
]].values

xyz(features, true_labels, n_cluster=N_CLUSTER)

### 4º Study Case - "Oportunidades" Indicators

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[[
    "Diversidade Partidária",
    "Transporte Público",

    "Acesso à cultura, esporte e lazer",
    "Gravidez na infância e adolescência",
    "Trabalho Infantil",
    "Vulnerabilidade familiar",
    
    "Violência contra indígenas",
    "Violência contra indígenas Taxa",
    "Violência contra mulheres",
    "Violência infantil",
    "Violência infantil Taxa",

    "Empregos ensino superior",
    "Mulheres com empregos ensino superior"
]].values

xyz(features, true_labels, n_cluster=N_CLUSTER)

### 5º Study Case -

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = LabelEncoder().fit_transform(dsyear["riscofogocat"].values)
features = dsyear[:1].values
features
# xyz(features, true_labels, n_cluster=N_CLUSTER)

In [ ]:
dsyear = dataset[dataset["ano"] == YEAR]
true_labels = LabelEncoder().fit_transform(dsyear["riscofogocat"].values)

indicators = [
    "Mortalidade infantil até 5 anos",
    "Mortalidade materna",
    "Mortalidade por desnutrição",
    "Mortalidade por doenças infecciosas",
    "Subnutrição",
    
    "Abastecimento de água",
    "Esgotamento sanitário",
    "Índice de atendimento de agua",
    
    "Coleta de lixo",
    "Moradias com iluminação adequada",
    "Moradias com parede adequada",
    "Moradias com piso adequado",
    
    "Assassinatos de jovens",
    "Assassinatos de jovens Taxa",
    "Homicídios",
    "Homicídios Taxa",
    "Mortes por acidente no trânsito",

    
    
    "Abandono escolar ensino fundamental",
    "Distorção idade-série ensino fundamental",
    "Distorção idade-série ensino médio",
    "Qualidade da educação Ideb ensino fundamental",
    "Reprovação escolar ensino fundamental",
    
    "Densidade internet banda-larga",
    "Densidade telefonia fixa",
    "Densidade telefonia movel",
    "Densidade TV por assinatura",
    
    "Mortalidade por diabetes mellitus",
    "Mortalidade por câncer",
    "Mortalidade por doenças circulatórias",
    "Mortalidade por doenças respiratórias",
    "Mortalidade por suicídios",

    "Áreas Protegidas",
    "Desmatamento acumulado",
    "Desmatamento recente",
    "Emissões CO2",
    "Focos de calor por habitantes",

    
    
    "Diversidade Partidária",
    "Transporte Público",

    "Acesso à cultura, esporte e lazer",
    "Gravidez na infância e adolescência",
    "Trabalho Infantil",
    "Vulnerabilidade familiar",
    
    "Violência contra indígenas",
    "Violência contra indígenas Taxa",
    "Violência contra mulheres",
    "Violência infantil",
    "Violência infantil Taxa",

    "Empregos ensino superior",
    "Mulheres com empregos ensino superior"
]

combinations = permutations(indicators, 5)

x = 0.0
for combination  in list(combinations):
    features = dsyear[list(combination)].values
    ari = abc(features, true_labels, n_cluster=N_CLUSTER)
    
    if ari > x:
        x = ari
        print(ari, combination)

0.01 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Mortalidade por doenças infecciosas', 'Subnutrição')
0.02 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Mortalidade por doenças infecciosas', 'Abastecimento de água')
0.05 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Mortalidade por doenças infecciosas', 'Moradias com iluminação adequada')
0.06 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Mortalidade por doenças infecciosas', 'Distorção idade-série ensino médio')
0.07 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Moradias com iluminação adequada', 'Distorção idade-série ensino médio')
0.08 ('Mortalidade infantil até 5 anos', 'Mortalidade materna', 'Mortalidade por desnutrição', 'Distorção idade-série ensino médio', 'Densidade TV por assinatura')
0.09 ('Mortalidad